<a href="https://colab.research.google.com/github/VeronicaHeringer/GEE/blob/main/NDWI_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar as bibliotecas necessárias
#!pip list
!pip install geemap earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.3 MB/s eta 0:00:00


In [2]:
# Importar as bibliotecas
import geemap
import ee

In [4]:
# Auntentique o API do GEE
ee.Authenticate()
# Inicialize o API do GEE
ee.Initialize(project='ee-vevelsf2')

In [5]:
# Definir a região de interesse
roi=ee.Geometry.Point([-55.392264164434856, -14.839981457233737])

# Definir a coleção de imagens
ee.collection=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                           .filterBounds(roi)\
                           .filterDate("2015-01-01", "2025-01-01")\
                           .filter(ee.Filter.lt("CLOUD_COVER", 1))


print("Qtds", ee.collection.size().getInfo())

Qtds 52


In [6]:
# Criar uma função pra calcular o NDWI
def ndwi(img):
  # Definir o fator de escala bandas opticas
  opticalBands=img.select("SR_B.").multiply(0.0000275).add(-0.2)
  # Definir o fator de escala bandas termais
  thermalBands=img.select("ST_B.*").multiply(0.00341802).add(149.0)
  # NDWI
  ndwi_image=img.normalizedDifference(["SR_B3", "SR_B5"]).rename("NDWI")
  # Aplicando e adicionandoas bandas com o fator de escala
  return img.addBands(opticalBands, None, True).addBands(thermalBands, None, True)\
            .addBands(ndwi_image).copyProperties(img, ["system:time_start"])\
            .set("date", img.date().format("YYYY-MM-dd"))

In [7]:
# Aplicando o fator de escala e ndwi em todas as imagens
collection_ndwi=ee.collection.map(ndwi)
print("Qtds imagens", collection_ndwi.size().getInfo())

Qtds imagens 52


In [8]:
collection_ndwi

In [9]:
print(collection_ndwi.first().bandNames().getInfo())

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT', 'NDWI']


In [10]:
# Criar máscaras pra identificar valores máximos e médios
mask_max=collection_ndwi.select("NDWI").max().gt(0).selfMask()
mask_mean=collection_ndwi.select("NDWI").mean().gt(0).selfMask()

In [12]:
# Defina um método para exibir os blocos de imagem do Earth Engine em um Folium map
Map=geemap.Map()

# Centralizando a imagem
Map=geemap.Map(location=[-30.2 , -54.93], zoon_start=10)

# Para ter uma visualização de satélite do Google como um mapa base
#Map.add_basemap("HYBRID")
Map.add_ee_layer(collection_ndwi.median(),{'bands':['SR_B4', 'SR_B3', 'SR_B2'],
                                           'min':0.03, 'max':0.19}, 'RGB')

# Adicionar os layers com as máscaras criadas
Map.add_ee_layer(mask_max,{'palette':['blue'], 'min':0, "max":1}, "NDWI_max")
Map.add_ee_layer(mask_mean,{'palette':['cyan'], 'min':0, "max":1}, "NDWI")

display(Map)

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [28]:
# Definir área de interesse (retangulo) para exportar
area=ee.Geometry.Polygon ( [[[ -55.77006835937501 , -14.840259649945242 ] ,
                             [ -55.77006835937501 , -15.17850104599419 ] ,
                             [ -55.367694091796885 , -15.17850104599419 ] ,
                             [ -55.367694091796885 , -14.840259649945242 ]]] , geodesic=False )



In [29]:
# Exportar as máscaras de resultado do NDWI em shapes

# Converter a máscara binária em vetor (polígonos)
vetor_1 = mask_max.reduceToVectors(
    geometry=area,
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,
    maxPixels=1e8)



In [30]:
# Exportar como Shapefile
task = ee.batch.Export.table.toDrive(
    collection=vetor_1,
    description='NDWI_Vector1',
    folder='GEE_Exports',
    fileNamePrefix='ndwi_vector1',
    fileFormat='SHP')
task.start()

In [31]:
# Continuar a exportar o shp para a mask_mean

# Converter a máscara binária em vetor (polígonos)
vetor_2 = mask_mean.reduceToVectors(
    geometry=area,
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,
    maxPixels=1e8)

# Exportar como Shapefile
task = ee.batch.Export.table.toDrive(
    collection=vetor_2,
    description='NDWI_Vector2',
    folder='GEE_Exports',
    fileNamePrefix='ndwi_vector2',
    fileFormat='SHP')
task.start()